<a href="https://colab.research.google.com/github/alexandrufalk/Background-generator/blob/Layout-page/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The main goal of natural language processing (NLP) is to derive information from natural language.

Natural language is a broad term but you can consider it to cover any of the following:

Text (such as that contained in an email, blog post, book, Tweet)
Speech (a conversation you have with a doctor, voice commands you give to a smart speaker)

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-06-04 09:05:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-06-04 09:05:56 (61.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

#Download a text dataset

In [4]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-06-04 09:06:11--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2024-06-04 09:06:11 (52.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many samples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
len(train_df)-5

7608

In [10]:
# Let's visualize some random training examples
import random
random_index=random.randint(0,len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Do babies actually get electrocuted from wall sockets? I'm wondering how I and those before me survived childhood.

---

Target: 0 (not real disaster)
Text:
Choking Hazard Prompts Recall Of Kraft Cheese Singles http://t.co/XGKyVF9t4f

---

Target: 0 (not real disaster)
Text:
Reddit Will Now Quarantine Offensive Content http://t.co/u9BkQt6XHR

---

Target: 1 (real disaster)
Text:
300000 Dead 1200000 injured 11000000 Displaced this is #Syria 2015 visit  http://t.co/prCI76hOwu  #US #Canada http://t.co/rTCuFaG0au

---

Target: 0 (not real disaster)
Text:
@Reds I don't throw the word hero around lightly... Usually reserved for first responders and military... But he's a hero...

---



#Split data into training and validation sets


In [11]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                        train_df_shuffled["target"].to_numpy(),
                                                                        test_size=0.1, # dedicate 10% of samples to validation set
                                                                        random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)


(6851, 6851, 762, 762)

In [13]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]


(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

In NLP, there are two main concepts for turning text into numbers:

**Tokenization** - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:

Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.

Character-level tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.

Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.

**Embeddings **- An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings:

Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.

Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer=TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                  standardize="lower_and_strip_punctuation", # how to process text
                                  split="whitespace",# how to split tokens
                                  ngrams=None, # create groups of n-words?
                                  output_mode="int", # how to map tokens to numbers
                                  output_sequence_length=None # how long should the output sequence of tokens be?
                                  )

In [15]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# Setup text vectorization with custom variables
max_vocab_length=10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  output_mode='int',
                                  output_sequence_length=max_length)


In [17]:
# Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it

random_sentance=random.choice(train_sentences)
print(f"original text:\n{random_sentance}\
      \n\n Vectorize version:")
text_vectorizer([random_sentance])

original text:
@DelDryden If I press on the twitch will my head explode?      

 Vectorize version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,   47,    8, 1265,   11,    2, 7283,   38,   13,  331,  332,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#Creating an Embedding using an Embedding Layer

We can see what an embedding of a word looks like by using the tf.keras.layers.Embedding layer.

Creating an Embedding using an Embedding Layer
We've got a way to map our text to numbers. How about we go a step further and turn those numbers into an embedding?

The powerful thing about an embedding is it can be learned during training. This means rather than just being static (e.g. 1 = I, 2 = love, 3 = TensorFlow), a word's numeric representation can be improved as a model goes through data samples.

We can see what an embedding of a word looks like by using the tf.keras.layers.Embedding layer.

The main parameters we're concerned about here are:

input_dim - The size of the vocabulary (e.g. len(text_vectorizer.get_vocabulary()).

output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

input_length - Length of sequences being passed to embedding layer.


In [21]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, #set size of embeding vector
                           embeddings_initializer="uniform",  # default, intialize randomly
                           input_length=max_length, #how long is each input
                           name="embedding_1"
                           )
embedding

In [22]:
# Get a random sentence from training set

random_sentence=random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Alarming Rise in Dead Marine Life Since the #Fukushima Nuclear Disaster: http://t.co/v6H97K688J http://t.co/tJw9bSeiPW      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04230013,  0.03615511,  0.0101255 , ...,  0.02031727,
          0.04199744, -0.02442577],
        [ 0.01233441,  0.01029553,  0.04475648, ...,  0.00111137,
         -0.03824825, -0.04062319],
        [ 0.04646926,  0.04307102,  0.0009881 , ..., -0.03138378,
         -0.00132352, -0.04304807],
        ...,
        [ 0.04230013,  0.03615511,  0.0101255 , ...,  0.02031727,
          0.04199744, -0.02442577],
        [-0.02675541,  0.0456002 ,  0.0425829 , ...,  0.04600369,
          0.03223851,  0.00485436],
        [-0.02675541,  0.0456002 ,  0.0425829 , ...,  0.04600369,
          0.03223851,  0.00485436]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.04230013,  0.03615511,  0.0101255 ,  0.01669789,  0.01775176,
        0.02359347, -0.0364549 , -0.03560208,  0.03197123,  0.04827717,
       -0.03023901, -0.01343624, -0.02396501,  0.03533455,  0.0355876 ,
        0.0014888 , -0.00658928, -0.00314377, -0.01224812, -0.02594951,
        0.01590793, -0.01358627,  0.00210763,  0.04130897, -0.02516035,
        0.0276466 ,  0.03705212, -0.01304892, -0.01644844, -0.01599498,
        0.00155495,  0.02604289,  0.04505492, -0.02588108,  0.0016362 ,
       -0.04516499, -0.04415938,  0.03206417,  0.02320101, -0.03765883,
       -0.02329286,  0.02281192, -0.02475805,  0.02516503,  0.00447719,
       -0.04267895, -0.0236794 ,  0.02101887,  0.02298551,  0.01808392,
       -0.0368126 ,  0.00790953,  0.03372643, -0.02671366,  0.04679606,
        0.0467256 , -0.04046738,  0.04855952, -0.02249335, -0.04114873,
       -0.04968592, -0.02544792, -0.01369898,  0.038509  ,  0.04660196,
       -0.012379

#Modelling a text dataset

**Model 0**: Naive Bayes (baseline)

**Model 1**: Feed-forward neural network (dense model)

**Model 2**: LSTM model

**Model 3**: GRU model

**Model 4**: Bidirectional-LSTM model

**Model 5**: 1D Convolutional Neural Network

**Model 6**: TensorFlow Hub Pretrained Feature Extractor

**Model 7**: Same as model 6 with 10% of training data


#Model 0: Getting a baseline

To create our baseline, we'll create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert our words to numbers and then model them with the Multinomial Naive Bayes algorithm. This was chosen via referring to the Scikit-Learn machine learning map.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0=Pipeline([
                  ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                  ("clf", MultinomialNB()) #model the text
                  ])

## Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score=model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#Creating an evaluation function for our model experiments

In [27]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


In [28]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#Model 1: A simple dense model

In [29]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [30]:
#Build model with the Functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x=text_vectorizer(inputs) #tunrn the input text into number
x=embedding(x) #create an embedding of the numerized nubers
x=layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs=layers.Dense(1,activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1=tf.keras.Model(inputs, outputs, name="model_1_dense") #construct the model


In [31]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [32]:
#Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [33]:
# Fit the model
model_1_history=model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in mode
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20240604-090618
Epoch 1/5
215/215 [==============================] - 16s 61ms/step - loss: 0.6079 - accuracy: 0.6932 - val_loss: 0.5345 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.4399 - accuracy: 0.8205 - val_loss: 0.4686 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 7s 33ms/step - loss: 0.3453 - accuracy: 0.8624 - val_loss: 0.4586 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 6s 28ms/step - loss: 0.2837 - accuracy: 0.8932 - val_loss: 0.4639 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 6s 28ms/step - loss: 0.2368 - accuracy: 0.9129 - val_loss: 0.4765 - val_accuracy: 0.7861


In [34]:
# Check the results
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4765 - accuracy: 0.7861


[0.4764637351036072, 0.7860892415046692]

In [35]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.00549696,  0.02518932,  0.06425539, ...,  0.06822587,
          0.01170498,  0.02603291],
        [ 0.04395263,  0.03408699,  0.01191824, ...,  0.01966528,
          0.0403648 , -0.02470219],
        [ 0.02774628,  0.03124247, -0.02536817, ...,  0.06727652,
         -0.04134562,  0.03210051],
        ...,
        [ 0.04831001, -0.02984275,  0.00205853, ..., -0.03372871,
         -0.0054462 ,  0.04805077],
        [ 0.02915069, -0.00480597,  0.07834543, ...,  0.03017418,
         -0.02032201,  0.08392763],
        [ 0.09923094, -0.03640844,  0.05191839, ...,  0.08965793,
         -0.05360966,  0.06240108]], dtype=float32)>]

In [36]:
embeded_weights=model_1.get_layer("embedding_1").get_weights()[0]
print(embeded_weights.shape)

(10000, 128)


In [37]:
# # View tensorboard logs of transfer learning modelling experiments (should be 4 models)
# # Upload TensorBoard dev records
# !tensorboard dev upload --logdir ./model_logs \
#   --name "First deep model on text data" \
#   --description "Trying a dense model with an embedding layer" \
#   --one_shot # exits the uploader when upload has finished

In [38]:
# If you need to remove previous experiments, you can do so using the following command
# !tensorboard dev delete --experiment_id EXPERIMENT_ID_TO_DELETE

In [39]:
# Make predictions (these come back in the form of probabilities)

model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.42042485],
       [0.7464144 ],
       [0.99764997],
       [0.10953413],
       [0.10489663],
       [0.93587184],
       [0.9127023 ],
       [0.9927611 ],
       [0.9677019 ],
       [0.26236114]], dtype=float32)

In [40]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs)) #squeeze remove single dimensions
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [41]:
# Calculate model_1 metrics
model_1_results=calculate_results(y_true=val_labels,
                                  y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [42]:
# Is our simple Keras model better than our baseline model?
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

In [43]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key,value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


#Visualizing learned embeddings

Once you've downloaded the embedding vectors and metadata, you can visualize them using Embedding Vector tool:

Go to http://projector.tensorflow.org/

Click on "Load data"

Upload the two files you downloaded (embedding_vectors.tsv and embedding_metadata.tsv)

Explore

Optional: You can share the data you've created by clicking "Publish"


In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0:
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

#Recurrent Neural Networks (RNN's)
The premise of an RNN is simple: use information from the past to help you with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

This concept is especially helpful when dealing with sequences such as passages of natural language text (such as our Tweets).

ecurrent neural networks can be used for a number of sequence-based problems:

One to one: one input, one output, such as image classification.

One to many: one input, many outputs, such as image captioning (image input, a sequence of text as caption output).

Many to one: many inputs, one outputs, such as text classification (classifying a Tweet as real diaster or not real diaster).

Many to many: many inputs, many outputs, such as machine translation (translating English to Spanish) or speech to text (audio wave as input, text as output).

When you come across RNN's in the wild, you'll most likely come across variants of the following:

Long short-term memory cells (LSTMs).

Gated recurrent units (GRUs).

Bidirectional RNN's (passes forward and backward along a sequence, left to right and right to left).



 Resources:

MIT Deep Learning Lecture on Recurrent Neural Networks - explains the background of recurrent neural networks and introduces LSTMs. https://youtu.be/SEnXr6v2ifU

The Unreasonable Effectiveness of Recurrent Neural Networks by Andrej Karpathy - demonstrates the power of RNN's with examples generating various sequences.
http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Understanding LSTMs by Chris Olah - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.
https://colah.github.io/posts/2015-08-Understanding-LSTMs/


#Model 2: LSTM

tensorflow.keras.layers.LSTM().

Our model is going to take on a very similar structure to model_1:

Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)

In [50]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")
#Create LSTM model
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x=layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")





(None, 15, 128)
(None, 64)


In [51]:
#Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [52]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [54]:
#Fit the model
model_2_history=model_2.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                   "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20240604-093036
Epoch 1/5
215/215 [==============================] - 17s 53ms/step - loss: 0.5069 - accuracy: 0.7440 - val_loss: 0.4606 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 14s 66ms/step - loss: 0.3142 - accuracy: 0.8737 - val_loss: 0.5203 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 13s 62ms/step - loss: 0.2167 - accuracy: 0.9165 - val_loss: 0.6077 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 14s 67ms/step - loss: 0.1494 - accuracy: 0.9491 - val_loss: 0.6737 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 14s 67ms/step - loss: 0.1052 - accuracy: 0.9599 - val_loss: 0.7661 - val_accuracy: 0.7598


In [55]:
# Make predictions on the validation dataset
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]  # view the first 10

24/24 [==============================] - 1s 9ms/step


((762, 1),
 array([[0.02471067],
        [0.7900822 ],
        [0.9984067 ],
        [0.05599919],
        [0.00531065],
        [0.9993283 ],
        [0.9348848 ],
        [0.99961877],
        [0.9994435 ],
        [0.1642556 ]], dtype=float32))

In [56]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [57]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.98425196850394,
 'precision': 0.7618096125081139,
 'recall': 0.7598425196850394,
 'f1': 0.7573149475055201}